<a href="https://colab.research.google.com/github/Pathway2008/stock_portfolio_krx/blob/main/pykrx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 액면분할은 주가하락으로 평가
2. 모든 주식은 동일금액(수익률이 중요)
3. 거래중지 종목들이 있음(5월 30일에 거래하지 못하는 종목은 제외해야함)
4. 평가기간에 액면분할되는 종목은 평가가 어떻게 되는지 확인할 것(문의 = 액면분할 - 주가하락, 액면병합 - 주가상승)

In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/portfolio/train.csv')
sub = pd.read_csv('/content/drive/MyDrive/portfolio/sample_submission.csv')

In [ ]:
train.head()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 988000 entries, 0 to 987999
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   일자      988000 non-null  int64 
 1   종목코드    988000 non-null  object
 2   종목명     988000 non-null  object
 3   거래량     988000 non-null  int64 
 4   시가      988000 non-null  int64 
 5   고가      988000 non-null  int64 
 6   저가      988000 non-null  int64 
 7   종가      988000 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 60.3+ MB


In [ ]:
train.describe()

,일자,거래량,시가,고가,저가,종가
count,9.880000e+05,9.880000e+05,9.880000e+05,9.880000e+05,9.880000e+05,9.880000e+05
mean,2.021974e+07,7.734265e+05,2.587701e+04,2.634938e+04,2.542335e+04,2.600283e+04
std,6.822396e+03,4.839914e+06,6.590845e+04,6.689822e+04,6.490307e+04,6.587082e+04
min,2.021060e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.500000e+01
25%,2.021113e+07,3.653900e+04,3.775000e+03,3.865000e+03,3.700000e+03,3.890000e+03
50%,2.022053e+07,1.140220e+05,8.630000e+03,8.810000e+03,8.470000e+03,8.700000e+03
75%,2.022113e+07,3.672998e+05,2.130000e+04,2.180000e+04,2.085000e+04,2.140000e+04
max,2.023053e+07,7.822434e+08,1.770000e+06,1.784000e+06,1.756000e+06,1.770000e+06


In [ ]:
for i in range(986000):
    opening_yesterday = train.loc[i, '시가']
    closing_current = train.loc[i+2000, '종가']

    yield_value = (closing_current - opening_yesterday) / opening_yesterday
    train.loc[i+2000, 'yield'] = yield_value


<ipython-input-13-4e48214d13d4>:5: RuntimeWarning: divide by zero encountered in long_scalars
  yield_value = (closing_current - opening_yesterday) / opening_yesterday


In [ ]:
train.tail()

In [ ]:
train.to_csv('train_yield.csv', index=False)

15/30/45D 평균이동선

In [ ]:
abc_rows = train[train['종목명'] == train['종목명'][0]].copy()
abc_rows.reset_index(drop=True, inplace=True)
abc_rows.head()

,일자,종목코드,종목명,거래량,시가,고가,저가,종가,yield
0,20210601,A060310,3S,166690,2890,2970,2885,2920,NaN
1,20210602,A060310,3S,134833,2915,2975,2830,2900,0.003460
2,20210603,A060310,3S,144470,2900,2925,2875,2900,-0.005146
3,20210604,A060310,3S,934224,2930,3120,2920,2950,0.017241
4,20210607,A060310,3S,946560,3000,3150,2955,3150,0.075085


In [ ]:
new_df = pd.DataFrame()

In [ ]:
for i in range(1999):
  for j in range(478):
    avg = abc_rows.loc[j+1:j+15, 'yield'].mean()
    new_df.loc[j, '15D_'+train['종목명'][i]] = avg

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 0 to 477
Columns: 1999 entries, 15D_3S to 15D_희림
dtypes: float64(1999)
memory usage: 7.3 MB


In [ ]:
new_df.to_csv('15D_yield.csv', index=False)

In [ ]:
for i in range(1999):
  for j in range(463):
    avg = abc_rows.loc[j+1:j+30, 'yield'].mean()
    new_df.loc[j, '30D_'+train['종목명'][i]] = avg

In [ ]:
for i in range(1999):
  for j in range(448):
    avg = abc_rows.loc[j+1:j+45, 'yield'].mean()
    new_df.loc[j, '45D_'+train['종목명'][i]] = avg

In [ ]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 0 to 477
Columns: 5997 entries, 15D_3S to 45D_희림
dtypes: float64(5997)
memory usage: 21.9 MB


In [ ]:
new_df.to_csv('15D_30D_45D_yield.csv', index=False)

액면분할or액면합병된 주식들

In [ ]:
split_merger = pd.DataFrame(columns=['Type', '일자', '종목명','종가'])

for i in range(986000):
    closing_yesterday = train.loc[i, '종가']
    closing_current = train.loc[i+2000, '종가']
    if closing_yesterday * 1.5 <= closing_current:
        split_merger = split_merger.append({'Type': '액면합병', '일자': train.loc[i, '일자'], '종목명': train.loc[i, '종목명'],'종가': train.loc[i, '종가']}, ignore_index=True)
    elif closing_current * 1.5 <= closing_yesterday:
        split_merger = split_merger.append({'Type': '액면분할', '일자': train.loc[i, '일자'], '종목명': train.loc[i, '종목명'],'종가': train.loc[i, '종가']}, ignore_index=True)
    else:
        pass

print(split_merger)

In [ ]:
split_merger

,Type,일자,종목명,종가
0,액면분할,20210602,레이,53100
1,액면분할,20210602,영화테크,21900
2,액면분할,20210614,알리코제약,14050
3,액면분할,20210617,넥스턴바이오,17250
4,액면합병,20210629,웨스트라이즈,310
...,...,...,...,...
196,액면합병,20230515,대한전선,1492
197,액면합병,20230519,TYM,2490
198,액면합병,20230522,국보,570
199,액면분할,20230524,나우IB,12400


In [ ]:
split_merger['종목명'].value_counts()

어스앤에어로스페이스    3
이엠앤아이         2
CBI           2
테크엘           2
에스맥           2
             ..
이오플로우         1
판타지오          1
에이프로          1
대창단조          1
나우IB          1
Name: 종목명, Length: 184, dtype: int64

In [ ]:
value_counts = split_merger['종목명'].value_counts()
extracted_values = value_counts[value_counts >= 2].index.tolist()
extracted_values

['어스앤에어로스페이스',
 '이엠앤아이',
 'CBI',
 '테크엘',
 '에스맥',
 '메디콕스',
 '하이소닉',
 'KG모빌리티',
 'MDS테크',
 'ITX-AI',
 '에스엠벡셀',
 '이브이첨단소재',
 '모트렉스',
 '큐브엔터',
 '넥스턴바이오',
 '흥아해운']

In [ ]:
extracted_rows = split_merger[split_merger['종목명'].isin(extracted_values)]
extracted_rows

,Type,일자,종목명,종가
3,액면분할,20210617,넥스턴바이오,17250
5,액면합병,20210701,큐브엔터,4625
7,액면합병,20210707,흥아해운,258
13,액면분할,20210721,큐브엔터,27850
14,액면분할,20210727,CBI,22750
23,액면분할,20210824,이브이첨단소재,4950
34,액면합병,20210914,흥아해운,1035
40,액면합병,20211012,어스앤에어로스페이스,2015
41,액면합병,20211015,에스엠벡셀,306
50,액면합병,20211112,이엠앤아이,1390


In [ ]:
a = train[(train['종목명'] == '넥스턴바이오') & (train['일자'] == 20210618)]
a

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
26339,20210618,A089140,넥스턴바이오,2563676,6740,6900,6380,6650


In [ ]:
a = train[(train['종목명'] == '하이소닉') & (train['일자'] == 20230328)]
a

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
903799,20230328,A106080,하이소닉,2826564,11780,11780,8250,8250


관리종목 주식들 (거래중지)

5/30일에 관리종목이면 포트폴리오에 제외해야 함

거래중지가 된 종목     stock_names    205개

시작부터 거래중지 종목 stock_names_f   48개

**5/30일 거래중지 종목   stock_names_l   50개**

전체기간 거래중지 종목 all             14개

기간 중 거래중지 종목  stock_names_m  121개

In [ ]:
zero_closing_price_rows = train[train['시가'] == 0]

In [ ]:
stock_names = zero_closing_price_rows['종목명'].unique()
stock_names # len(stock_names) = 205

array(['CNT85', 'DXVX', 'ITX-AI', 'KG모빌리티', '강원에너지', '골드앤에스', '내츄럴엔도텍',
       '녹원씨엔아이', '대양제지', '디아크', '라이트론', '명성티엔에스', '비투엔', '성안', '세림B&G',
       '세원물산', '세원정공', '세코닉스', '스킨앤스킨', '신라젠', '쎌마테라퓨틱스', '아시아나IDT',
       '아시아나항공', '어스앤에어로스페이스', '에스에이티이엔지', '에스엠벡셀', '에어부산', '엔피', '엠젠솔루션',
       '와이투솔루션', '웰킵스하이텍', '이엠앤아이', '이큐셀', '일월지엠엘', '좋은사람들', '청호ICT',
       '커머스마이너', '컨버즈', '코센', '코스온', '큐리언트', '태웅', '테크엘', '프로이천', '하이딥',
       '하이소닉', '휴엠앤씨', '흥아해운', '에스앤디', 'HDC랩스', '스마트솔루션즈', '웨스트라이즈',
       '큐브엔터', '인카금융서비스', 'CBI', '블리츠웨이', '씨엔알리서치', '앤디포스', 'HD현대인프라코어',
       '오리엔트바이오', '한솔아이원스', '하인크코리아', '누보', '삼성중공업', '이즈미디어', '파이버프로',
       'HLB이노베이션', '쿠쿠홀딩스', '제주항공', '휴먼엔', '한일시멘트', '디와이디', '모비데이즈',
       '나노씨엠에스', '세종메디칼', '웨이버스', '남양유업', 'FSN', '멜파스', '에코캡', '원텍',
       '초록뱀이앤엠', '마니커', '프로테옴텍', '엠투엔', '위메이드맥스', 'SK텔레콤', '휴센텍', '베셀',
       '광무', '디지캡', '제노코', '아이엠', '인트로메딕', '코이즈', '오스템임플란트', '판타지오', '태성',
       '엔투텍', '대한전선', '에스맥', '대창단조', '세토피아', '계양전기', '이노시스', '안트로젠',
  

In [ ]:
count_f = 0
stock_names_f = set()

for i in range(2000):
    zero_price = train['시가'][i]
    if zero_price == 0:
        stock_name = train.loc[i, '종목명']
        print(train.loc[i, '종목명'])
        stock_names_f.add(stock_name)
        count_f += 1
    else:
        pass

print("Number of outputs:", count_f) #48

CNT85
DXVX
ITX-AI
KG모빌리티
강원에너지
골드앤에스
내츄럴엔도텍
녹원씨엔아이
대양제지
디아크
라이트론
명성티엔에스
비투엔
성안
세림B&G
세원물산
세원정공
세코닉스
스킨앤스킨
신라젠
쎌마테라퓨틱스
아시아나IDT
아시아나항공
어스앤에어로스페이스
에스에이티이엔지
에스엠벡셀
에어부산
엔피
엠젠솔루션
와이투솔루션
웰킵스하이텍
이엠앤아이
이큐셀
일월지엠엘
좋은사람들
청호ICT
커머스마이너
컨버즈
코센
코스온
큐리언트
태웅
테크엘
프로이천
하이딥
하이소닉
휴엠앤씨
흥아해운
Number of outputs: 48


In [ ]:
count_l = 0
stock_names_l = set()

for i in reversed(range(len(train) - 2000, len(train))):
    zero_price = train['시가'][i]
    if zero_price == 0:
        stock_name = train.loc[i, '종목명']
        print(train.loc[i, '종목명'])
        stock_names_l.add(stock_name)
        count_l += 1
    else:
        pass

print("Number of outputs:", count_l) #50

휴센텍
휴먼엔
현대약품
한송네오텍
한국테크놀로지
피에이치씨
포인트모바일
티엘아이
코스온
컨버즈
커머스마이너
청호ICT
좋은사람들
조광ILI
일월지엠엘
인트로메딕
인바이오젠
이화전기
이트론
이큐셀
이즈미디어
이아이디
이수화학
이노시스
어스앤에어로스페이스
앤디포스
쎌마테라퓨틱스
시스웍
스마트솔루션즈
셀리버리
세원이앤씨
선도전기
비덴트
버킷스튜디오
명성티엔에스
멜파스
디아크
동국제강
대유
대양제지
뉴지랩파마
녹원씨엔아이
국일제지
광림
골드퍼시픽
KH 필룩스
ITX-AI
IHQ
DGP
CBI
Number of outputs: 50


In [ ]:
common_stock_names = stock_names_f.intersection(stock_names_l)
print("Common stock names:", common_stock_names)
print(len(common_stock_names)) # 14

Common stock names: {'일월지엠엘', '커머스마이너', '디아크', '쎌마테라퓨틱스', '컨버즈', '명성티엔에스', '녹원씨엔아이', '청호ICT', '좋은사람들', '대양제지', 'ITX-AI', '어스앤에어로스페이스', '이큐셀', '코스온'}
14


In [ ]:
stock_names_m = set(stock_names) - set(stock_names_f)-set(stock_names_l)
stock_names_m

{'CSA 코스믹',
 'DH오토웨어',
 'DI동일',
 'F&F',
 'FSN',
 'HDC랩스',
 'HD현대인프라코어',
 'HLB바이오스텝',
 'HLB이노베이션',
 'KC그린홀딩스',
 'MDS테크',
 'OCI홀딩스',
 'SK텔레콤',
 'TYM',
 '계양전기',
 '광무',
 '광주신세계',
 '국보',
 '그래디언트',
 '기산텔레콤',
 '나노씨엠에스',
 '나우IB',
 '남양유업',
 '넥스턴바이오',
 '누보',
 '다이나믹디자인',
 '대창단조',
 '대한전선',
 '동원F&B',
 '동원산업',
 '디와이디',
 '디지캡',
 '리더스 기술투자',
 '마니커',
 '메디콕스',
 '메타랩스',
 '모비데이즈',
 '모트렉스',
 '미래아이앤지',
 '바이온',
 '밸로프',
 '베셀',
 '블리츠웨이',
 '삼성중공업',
 '삼화전자',
 '서울제약',
 '세종메디칼',
 '세종텔레콤',
 '세토피아',
 '셀바스헬스케어',
 '소니드',
 '소프트센',
 '신대양제지',
 '신세계 I&C',
 '신세계인터내셔날',
 '신영와코루',
 '씨엔알리서치',
 '아세아시멘트',
 '아이에스이커머스',
 '아이엠',
 '아이윈플러스',
 '안트로젠',
 '알에프세미',
 '알엔투테크놀로지',
 '어반리튬',
 '에스맥',
 '에스씨엠생명과학',
 '에스앤디',
 '에스티큐브',
 '에이프로젠',
 '에이프로젠 H&G',
 '에코캡',
 '엔투텍',
 '엠투엔',
 '오리엔트바이오',
 '오성첨단소재',
 '오스템임플란트',
 '원텍',
 '웨스트라이즈',
 '웨이버스',
 '위메이드맥스',
 '유니드',
 '이브이첨단소재',
 '인디에프',
 '인카금융서비스',
 '일성신약',
 '일신바이오',
 '일신방직',
 '자이글',
 '제노코',
 '제주항공',
 '조선내화',
 '중앙디앤엠',
 '초록뱀미디어',
 '초록뱀이앤엠',
 '카나리아바이오',
 '카이노스메드',
 '코스나인',
 '코스텍시스',
 '코오롱글로벌',
 '코이즈',


In [ ]:
len(set(stock_names) - set(stock_names_f)-set(stock_names_l))

121

In [20]:
!pip install onnx==1.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for onnx (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for onnx
Failed to build onnx
ERROR: Could not build wheels for onnx, which is required to install pyproject.toml-based projects


In [19]:
!pip install cudf

  Using cached cudf-0.6.1.post1.tar.gz (1.1 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cudf
  Running setup.py clean for cudf
Failed to build cudf
ERROR: Could not build wheels for cudf, which is required to install pyproject.toml-based projects


In [4]:
!pip install pykrx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.4 MB/s eta 0:00:00


In [22]:
import pandas as pd
import time
from pykrx import stock
from tqdm import tqdm

# Load the stock codes and names from the CSV file
stock_codes = train['종목코드'][:2000].tolist()
stock_names = train['종목명'][:2000].tolist()

In [24]:
start_date = '2021-06-01'
end_date = '2023-05-30'

In [ ]:
stock_data = {}
progress_bar = tqdm(total=len(stock_codes), desc='Progress', unit='stock')

# Retrieve financial data for each stock
for code, name in zip(stock_codes, stock_names):
    try:
        df = stock.get_market_ohlcv_by_date(start_date, end_date, code)
        stock_data[name] = df
    except Exception as e:
        print(f"Error retrieving data for {name} ({code}): {str(e)}")
    time.sleep(1)
    progress_bar.update(1)
progress_bar.close()

In [ ]:
example_stock = stock_data['EXAMPLE STOCK']
print(example_stock)

In [ ]:
stock_data = {}

# Create a tqdm progress bar
progress_bar = tqdm(total=len(stock_codes), desc='Progress', unit='stock')

# Retrieve financial data for each stock
for code, name in zip(stock_codes, stock_names):
    try:
        # Use cudf to retrieve and process the OHLCV data
        df = stock.get_market_ohlcv_by_date(start_date, end_date, code)
        df['종목코드'] = code
        df['종목명'] = name
        df['등락가'] = df['종가'] - df['시가']
        df['등락률'] = (df['등락가'] / df['시가']) * 100

        # Use cudf to retrieve and process the fundamental data
        df_info = stock.get_market_fundamental_by_date(start_date, end_date, code)
        df_info['종목코드'] = code
        df_info['종목명'] = name

        # Merge the data using cudf
        df = df.merge(df_info, on=['날짜', '종목코드', '종목명']).to_pandas()

        stock_data[name] = df
    except Exception as e:
        print(f"Error retrieving data for {name} ({code}): {str(e)}")

    time.sleep(1)  # Delay for 1 second between each request
    progress_bar.update(1)  # Update the progress bar

# Close the progress bar
progress_bar.close()

Progress:   1%|▏         | 26/2000 [01:10<1:28:55,  2.70s/stock]


Error retrieving data for 3S (A060310): '종가'


Progress:   0%|          | 1/2000 [00:01<46:33,  1.40s/stock]

Error retrieving data for AJ네트웍스 (A095570): '종가'


Progress:   0%|          | 2/2000 [00:02<44:22,  1.33s/stock]

...

Error retrieving data for 컴퍼니케이 (A307930): '종가'


Progress:  77%|███████▋  | 1548/2000 [32:57<09:24,  1.25s/stock]

Error retrieving data for 케어랩스 (A263700): '종가'


Progress:  77%|███████▋  | 1549/2000 [32:58<09:21,  1.25s/stock]

Error retrieving data for 케어젠 (A214370): '종가'
